In [2]:
import os
import builderMethods
import distanceMethods
import costMethods
import cacheMethods
import csv

## Airtravel Optimization Program

### The below code is a our implementation of a program that calculates the lowest cost route for a aircraft to take. The user provides a csv file, that is read, line by line (containing the airports needed to go and aircrft used) and determines the optimal route to take in order to minimise the costs. 

### The base currency is euro. In order to refuel the aircraft, you must purchase the refuel the aircraft to full using the local currecy. One kilometer requires one unit of fuel and can be purchased with one unit of the local currency. This means that the gain and loss of the cost of fuel is in the conversion rate between euro and the local currency.

### To complete this exercise, we have implemented various data structres and algorithms. These will be explained as we make our way down the notebook.

## Preparing

### Below we have developed both functions and classes that are used at the beginning of the program. The following is a breakdown:
 
### 1. **Read file function:** The function reads in a csv file and creates a list for each line in the csv. Each line in the csv is then appended to a bigger datamatrix list. This list will contain the airports that are to be visited and the aircraft to be used.


In [3]:
def read_file(file_name):
    """Read data in the file and Create Matrix"""
    datamatrix = []
    datafile = open(file_name)
    for line in datafile:
        datamatrix.append(line.split(","))
    datafile.close()
    return datamatrix

## Check Aircraft Capability
### Time complexity: O(n^2 + n)

In [4]:
def checkAircraftAllowed(dictAirplane, distanceDict, input_list):
    """Checks that the aircraft being used can do the route. Returns the routes that are
    only possible with the aircraft"""
    planeToFly = input_list[5]
    planeRange = dictAirplane[planeToFly].max_capacity
    distanceDict_copy = distanceDict.copy()
    for j in distanceDict_copy:
        if distanceDict_copy[j] < int(planeRange):
            distanceDict.pop(j)
    finalRouteDict_copy = finalRouteDict.copy()
    for i in finalRouteDict_copy:
        toRemove = False
        for j in distanceDict:
            x = 0
            while x < len(i) - 1:
                if str(i[x] + "_" + i[x + 1]) == j:
                    toRemove = True
                x += 1
        if toRemove:
            finalRouteDict.pop(i)

In [11]:
cacheDict = {}
print("Please enter the name of the file containing the routes: ")
file_name = input("> ")
while not os.path.isfile(file_name):
    print("I am sorry. I do not believe that file exists. Remember, it is case sensitive. Please try again.")
    print("=" * 75)
    print("")
    file_name = input("> ")

print(f"Opening {file_name}....")
datamatrix = read_file(file_name)
i = 0
while i < len(datamatrix):
    inputList = datamatrix[i]  # sample list passed
    cacheCheck = cacheMethods.checkCache(inputList,cacheDict)
    if cacheCheck != False:
        resultList = [cacheCheck[0:-1]]
        resultList.append(cacheCheck[-1])
        with open('finalResults.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(resultList)
        
        csvFile.close()
        i += 1
        continue
    else:
        pass
    inputList[-1] = inputList[-1].rstrip()
    dictOfAircrafts = builderMethods.buildAircraft(inputList[-1])  # Creates aircraft objects
    if not dictOfAircrafts:
        print("The airplane specified does not exist, skipping this route.")
        i += 1
        continue
    airport_objects_dict = builderMethods.buildAirports(inputList)  # creates the objects for each airport
    if not airport_objects_dict:
        print("One of the airports specified does not exist, skipping this route.")
        i += 1
        continue

    # create all the possible routes
    all_routes_list = distanceMethods.allPerms(inputList)

    # Finds distances and costs of each leg
    dict_routes_distances = distanceMethods.leg_distance_calculator(inputList, airport_objects_dict)
    leg_costs = costMethods.findLegCosts(dict_routes_distances, airport_objects_dict)

    # Finds total route cost
    finalRouteDict = costMethods.findRouteCost(all_routes_list, leg_costs)

    # Removes distances that the aircraft cannot do
    checkAircraftAllowed(dictOfAircrafts, dict_routes_distances, inputList)
    if (len(finalRouteDict)) == 0:
        result = "This route is not possible with the " + str(inputList[-1]) + ". Try another plane. "
        resultList = [result]
        with open('finalResults.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(resultList)
        
        csvFile.close()
        i += 1
        continue
    cheapestRoute = min(finalRouteDict, key=finalRouteDict.get)
    cost = finalRouteDict[cheapestRoute]
    resultList = [cheapestRoute]
    resultList.append(cost)
    with open('finalResults.csv', 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(resultList)
        
    csvFile.close()
    cacheMethods.cacheRoutes(list(cheapestRoute), inputList[-1].rstrip(), cost, cacheDict)
    
    i += 1

print("Finished! The output is stored as finalResults in your working directory.")

Please enter the name of the file containing the routes: 
> test.csv
Opening test.csv....
Finished! The output is stored as finalResults in your working directory.
